In [ ]:
from preprocessing import getdataset, number_to_character

MAX_LEN = 282

dataset , table = getdataset("character", MAX_LEN)
print(table.number_to_character(0))

2024-12-26 22:04:07.208883: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-26 22:04:07.218758: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-26 22:04:07.230078: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-26 22:04:07.233542: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-26 22:04:07.242333: I tensorflow/core/platform/cpu_feature_guar

<pad>


In [ ]:
from network import make_model

EMBED_DIM = 32
NUM_HEADS = 8
num_transfo_blocs=  8
model = make_model(MAX_LEN, table.vocab_size, EMBED_DIM, NUM_HEADS,num_transfo_blocs)

I0000 00:00:1735247049.385854   19031 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1735247049.412186   19031 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1735247049.414369   19031 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1735247049.417476   19031 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 282)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, 282, 32)        │        16,544 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ (None, 282, 32)        │        34,752 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_1             │ (None, 282, 32)        │        34,752 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_2             │ (None, 282, 32)        │        34,752 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_3             │ (None, 282, 32)        │        34,752 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_4             │ (None, 282, 32)        │        34,752 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_5             │ (None, 282, 32)        │        34,752 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_6             │ (None, 282, 32)        │        34,752 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_7             │ (None, 282, 32)        │        34,752 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 282, 20)        │           660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 1, 20)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1, 235)         │         4,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 300,155 (1.15 MB)

 Trainable params: 300,155 (1.15 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["sparse_categorical_accuracy"])


In [5]:
X = tf.convert_to_tensor(dataset)
Y = np.zeros(X.shape).astype(int)
Y[:,0:-1] = np.array(X[:,1:])
Y = tf.convert_to_tensor(Y)
print(X)
print(Y)
#Y = tf.one_hot(Y, depth=table.vocab_size)
# change X to one_hot_vectors

tf.Tensor(
[[ 2  3  4 ...  0  0  0]
 [ 2 14 10 ...  0  0  0]
 [ 2 14 10 ...  0  0  0]
 ...
 [ 2 20 12 ...  0  0  0]
 [ 2 20 12 ...  0  0  0]
 [ 2 20 12 ...  0  0  0]], shape=(43352, 282), dtype=int32)
tf.Tensor(
[[ 3  4  5 ...  0  0  0]
 [14 10 11 ...  0  0  0]
 [14 10 11 ...  0  0  0]
 ...
 [20 12 16 ...  0  0  0]
 [20 12 16 ...  0  0  0]
 [20 12 16 ...  0  0  0]], shape=(43352, 282), dtype=int64)


In [6]:
history = model.fit(X, Y, epochs=20, batch_size=16)

Epoch 1/20


ValueError: Arguments `target` and `output` must have the same shape up until the last dimension: target.shape=(None, 282), output.shape=(None, 1, 235)

In [ ]:

text_to_begin_with = "@"
begining = [ table.table[i] for i in text_to_begin_with ]
nb_to_repeat = MAX_LEN - len(begining) -1
input = np.array([[2] + begining + [0] * nb_to_repeat])
exit = False
nb_iter = len(text_to_begin_with)+1
max_iter = MAX_LEN

print( "".join( [table.number_to_character( input[0,i] ) for i in range(len(input[0]))] ) )

while not exit:

  to_print = []
  for i in range(MAX_LEN):
    to_print.append(table.number_to_character(input[0, i]))

  input_tf = tf.convert_to_tensor(input)

  decoded = model.predict(input_tf)
  print(decoded.shape, input_tf.shape)
  output = []

  for i in range(MAX_LEN):
    output.append((np.argmax(decoded[0][i])))

  print("added : ")
  print( output )
  input[0, (nb_iter+1)] = output[nb_iter]

  if 1 in input or nb_iter >= max_iter-1:
    exit = True
  print("".join(to_print))

  nb_iter += 1

print("".join(to_print))




<sos><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>